# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [38]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sys
import re
import pickle
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, label_ranking_average_precision_score
from sklearn.model_selection  import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def load_data(db_path, table_name):
    """
    Load data from database, return X and y.
    Args:
      db_path(str): db file name included path
      table_name:(str): table name in the db file
    Return:
      X(pd.DataFrame): messages
      y(pd.DataFrame): labels for messages
    """
    
    # load data from database
    engine = create_engine('sqlite:///' + db_path)
    df = pd.read_sql_table(table_name, engine)
    
    # select columns for X and y
    X_cols = 'message'
    y_cols = [col for col in df.columns if re.search(pattern='-(0|1)', string=col)]
    
    # drop records with missing labels
    df = df.dropna(subset=y_cols, how='any')
    
    # X, y
    X = df[X_cols] #.values
    y = df[y_cols] #.values
    
    return X, y

In [3]:
X, y = load_data(db_path='etl_project.db', table_name='messages')

### 2. Write a tokenization function to process your text data

In [4]:
def lemmatize_all(text):
    
    lemmatizer = WordNetLemmatizer()
    
    tokens_lemmatized = []
    # create tokens from text
    for word, tag in pos_tag(word_tokenize(text)):
        # remove stop words from tokens
        if word not in stopwords.words('english'):
            
            # clean tokens
            if tag.startswith("NN"):
                token_lemmatized = lemmatizer.lemmatize(word, pos='n')
            elif tag.startswith('VB'):
                token_lemmatized = lemmatizer.lemmatize(word, pos='v')
            elif tag.startswith('JJ'):
                token_lemmatized = lemmatizer.lemmatize(word, pos='a')
            else:
                token_lemmatized = word
            
            tokens_lemmatized.append(token_lemmatized)
        
    return tokens_lemmatized

In [5]:
def tokenize(text):

    # normalize text
    text_normalized = re.sub(pattern=r'[^a-zA-Z0-9]', repl=' ', string=text.lower())
    
    # create tokens from text, remove stopwords from tokens, clean tokens
    tokens_lemmed = lemmatize_all(text_normalized)
    
    return tokens_lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
#TokenizerTransformer = FunctionTransformer(func=tokenize, validate=False)

In [7]:
# pipeline with RandomForestClassifier
pipeline_rf = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)), # CountVectorizer + TfidfTransformer
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200)))
])

In [8]:
# pipeline with KNeighborsClassifier
pipeline_knn = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)), # CountVectorizer + TfidfTransformer 
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

In [9]:
# pipeline with AdaBoostClassifier
pipeline_ab = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)), # CountVectorizer + TfidfTransformer 
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [10]:
# pipeline with DecisionTreeClassifier
pipeline_dt = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)), # CountVectorizer + TfidfTransformer 
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
pipeline_rf.fit(X_train[0:10], y_train[0:10])

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7f73305e6620>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=200)))])

In [13]:
pipeline_knn.fit(X_train[0:10], y_train[0:10])

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7f73305e6620>)),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

In [14]:
pipeline_ab.fit(X_train[0:10], y_train[0:10])

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7f73305e6620>)),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [15]:
pipeline_dt.fit(X_train[0:10], y_train[0:10])

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7f73305e6620>)),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [39]:
def performance_report_detailed(y_true, y_pred):
    
    y_cols = list(y_true.columns) 
    report = classification_report(y_true, y_pred, target_names=y_cols, zero_division=0)
    
    print(report)


def performance_report_summarised(y_true, y_pred):
    
    y_cols = list(y_true.columns) 
    report = classification_report(y_true, y_pred, target_names=y_cols, zero_division=0, output_dict=True)
    
    selected_keys = ['micro avg', 'macro avg', 'weighted avg', 'samples avg']
    report = {key: report[key] for key in selected_keys}
    report = pd.DataFrame.from_dict(report, orient='index')
    
    return report

In [51]:
print("Report - RandomForestClassifier\n")

y_train_pred = pipeline_rf.predict(X_train[0:10])
y_test_pred = pipeline_rf.predict(X_test[0:10])

print('Train performance:\n\n{}\n'.format(performance_report_summarised(y_train[0:10], y_train_pred)))
print('Test performance:\n\n{}'.format(performance_report_summarised(y_test[0:10], y_test_pred)))

Report - RandomForestClassifier

Train performance:

              precision    recall  f1-score  support
macro avg      0.416667  0.416667  0.416667       38
micro avg      1.000000  1.000000  1.000000       38
samples avg    0.800000  0.800000  0.800000       38
weighted avg   1.000000  1.000000  1.000000       38

Test performance:

              precision    recall  f1-score  support
macro avg      0.027778  0.037037  0.029630       17
micro avg      0.500000  0.352941  0.413793       17
samples avg    0.450000  0.281905  0.294444       17
weighted avg   0.235294  0.352941  0.266667       17


In [ ]:
# .descibe() -> mean accuracy etc.

In [52]:
print("Report - KNeighborsClassifier\n")

y_train_pred = pipeline_knn.predict(X_train[0:10])
y_test_pred = pipeline_knn.predict(X_test[0:10])

print('Train performance:\n\n{}\n'.format(performance_report_summarised(y_train[0:10], y_train_pred)))
print('Test performance:\n\n{}'.format(performance_report_summarised(y_test[0:10], y_test_pred)))

Report - KNeighborsClassifier

Train performance:

              precision    recall  f1-score  support
macro avg      0.083730  0.111111  0.095459       38
micro avg      0.760000  0.500000  0.603175       38
samples avg    0.675000  0.556667  0.532698       38
weighted avg   0.380827  0.500000  0.432122       38

Test performance:

              precision    recall  f1-score  support
macro avg      0.023148  0.046296  0.030864       17
micro avg      0.350000  0.411765  0.378378       17
samples avg    0.425000  0.315238  0.297778       17
weighted avg   0.205882  0.411765  0.274510       17


In [53]:
print("Report - AdaBoostClassifier\n")

y_train_pred = pipeline_ab.predict(X_train[0:10])
y_test_pred = pipeline_ab.predict(X_test[0:10])

print('Train performance:\n\n{}\n'.format(performance_report_summarised(y_train[0:10], y_train_pred)))
print('Test performance:\n\n{}'.format(performance_report_summarised(y_test[0:10], y_test_pred)))

Report - AdaBoostClassifier

Train performance:

              precision    recall  f1-score  support
macro avg      0.416667  0.416667  0.416667       38
micro avg      1.000000  1.000000  1.000000       38
samples avg    0.800000  0.800000  0.800000       38
weighted avg   1.000000  1.000000  1.000000       38

Test performance:

              precision    recall  f1-score  support
macro avg      0.020833  0.046296  0.028620       17
micro avg      0.350000  0.411765  0.378378       17
samples avg    0.375000  0.315238  0.264444       17
weighted avg   0.191176  0.411765  0.260250       17


In [54]:
print("Report - DecisionTreeClassifier\n")

y_train_pred = pipeline_dt.predict(X_train[0:10])
y_test_pred = pipeline_dt.predict(X_test[0:10])

print('Train performance:\n\n{}\n'.format(performance_report_summarised(y_train[0:10], y_train_pred)))
print('Test performance:\n\n{}'.format(performance_report_summarised(y_test[0:10], y_test_pred)))

Report - DecisionTreeClassifier

Train performance:

              precision    recall  f1-score  support
macro avg      0.416667  0.416667  0.416667       38
micro avg      1.000000  1.000000  1.000000       38
samples avg    0.800000  0.800000  0.800000       38
weighted avg   1.000000  1.000000  1.000000       38

Test performance:

              precision    recall  f1-score  support
macro avg      0.019290  0.050000  0.026984       17
micro avg      0.263158  0.294118  0.277778       17
samples avg    0.275000  0.267619  0.230794       17
weighted avg   0.145425  0.294118  0.191597       17


In [ ]:
pipeline.get_params()

### 6. Improve your model
Use grid search to find better parameters. 

In [59]:
parameters = {
    'clf__estimator__min_samples_leaf': [1, 10],
    'clf__estimator__max_features': ['auto', 'log2'],
    'vect__smooth_idf': [True]
}

cv_rf = GridSearchCV(pipeline_rf, param_grid=parameters, n_jobs=-1)

In [60]:
cv_rf.fit(X_train[0:10], y_train[0:10])

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x7f73305e6620>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=200)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__max_features': ['auto', 'log2'],
                         'clf__estimator__min_samples_leaf': [1, 10],
                         'vect__smooth_idf': [True]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print("Report - GridSearchCV x RandomForestClassifier\n")

y_train_pred = cv_rf.predict(X_train[0:10])
y_test_pred = cv_rf.predict(X_test[0:10])

print('Train performance:\n\n{}\n'.format(performance_report_summarised(y_train[0:10], y_train_pred)))
print('Test performance:\n\n{}'.format(performance_report_summarised(y_test[0:10], y_test_pred)))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
with open('classifer.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sys
import re
import pickle
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from sklearn.model_selection  import GridSearchCV

nltk.download(['punkt','stopwords','wordnet'])

In [ ]:
def load_data(db_path='workspace/data/DisasterResponse.db',tablename='disastertab'):
    """
    Function: load data from database and return X and y.
    Args:
      db_path(str): database file name included path
      tablename:(str): table name in the database file.
    Return:
      X(pd.DataFrame): messages for X
      y(pd.DataFrame): labels part in messages for y
    """
    # load data from database
    engine = create_engine('sqlite:///'+db_path)
    df=pd.read_sql('SELECT * from '+tablename, engine)
    X = df['message']
    # result is multiple classifty.
    y = df.iloc[:,4:]
    return X, y

def tokenize(text):
    """
    Function: tokenize the text
    Args:  source string
    Return:
    clean_tokens(str list): clean string list
    
    """
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

def build_model(): 
    """
    Function: build model that consist of pipeline
    Args:
      N/A
    Return
      cv(model): Grid Search model 
    """
    
    pipeline = Pipeline([
        ('vect',TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200,random_state=20)))
    ]) 
    
    parameters = {
        'clf__estimator__criterion':['entropy']
    }
    
    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
    return cv

def evaluate_model(y_test, y_pred):
    result=precision_recall_fscore_support(Y_test, y_pred)
    for i, col in enumerate(category_names):
        accu=accuracy_score(Y_test.loc[:,col],y_pred[:,i])
        print('{}\n Accuracy: {:.4f}    % Precision: {:.4f}   % Recall {:.4f} '.format(
            col,accu,result[0][i],result[1][i]))
    
def save_model(cv):
    """
    Function: save model as pickle file.
    Args:
      cv:target model
    Return:
      N/A
    """
    with open('classifer.pkl', 'wb') as f:
        pickle.dump(cv, f) 
        
def main(): 
    print("Load data")
    X,y=load_data()
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
    print("Build model")
    model=build_model()
    print("Train model")
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print("Evaluation model")
    evaluate_model(y_test, y_pred)
    print("Save model")
    save_model(model)

In [ ]:
main()